In [45]:
import sys
sys.path.append('/home/dcor/niskhizov/naturalspeech3_facodec/naturalspeech3_facodec')

In [47]:
pip install pyworld

DEPRECATION: Loading egg at /home/dcor/niskhizov/anaconda3/lib/python3.12/site-packages/diff_gauss-1.0.10.0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.5-cp312-cp312-linux_x86_64.whl size=197751 sha256=c93343a09c89238144f2c480bd654bbc778edbfdc785908c5f8dd27cdc0599e0
  Stored in directory: /home/dcor/niskhizov/.cache/pip/wheels/be/ac/58/c6a1791ec6d17f3a99b6ccdec92b472f560cb5c564b83dd77e
Successfully built pyworld
Note: you may need to restart the kernel to use updated packages.


In [48]:
from ns3_codec import FACodecEncoder, FACodecDecoder
from huggingface_hub import hf_hub_download


In [97]:
import librosa
import torch
import soundfile as sf
from ns3_codec import FACodecEncoderV2, FACodecDecoderV2

# Same parameters as FACodecEncoder/FACodecDecoder
# fa_encoder_v2 = FACodecEncoderV2(...)
# fa_decoder_v2 = FACodecDecoderV2(...)
fa_encoder_v2 = FACodecEncoderV2(
    ngf=32,
    up_ratios=[2, 4, 5, 5],
    out_channels=256,
)

fa_decoder_v2 = FACodecDecoderV2(
    in_channels=256,
    upsample_initial_channel=1024,
    ngf=32,
    up_ratios=[5, 5, 4, 2],
    vq_num_q_c=2,
    vq_num_q_p=1,
    vq_num_q_r=3,
    vq_dim=256,
    codebook_dim=8,
    codebook_size_prosody=10,
    codebook_size_content=10,
    codebook_size_residual=10,
    use_gr_x_timbre=True,
    use_gr_residual_f0=True,
    use_gr_residual_phone=True,
)

encoder_v2_ckpt = hf_hub_download(repo_id="amphion/naturalspeech3_facodec", filename="ns3_facodec_encoder_v2.bin")
decoder_v2_ckpt = hf_hub_download(repo_id="amphion/naturalspeech3_facodec", filename="ns3_facodec_decoder_v2.bin")

fa_encoder_v2.load_state_dict(torch.load(encoder_v2_ckpt))
fa_decoder_v2.load_state_dict(torch.load(decoder_v2_ckpt))

def load_audio(wav_path):
  wav = librosa.load(wav_path, sr=16000)[0]
  wav = torch.from_numpy(wav).float()
  wav = wav.unsqueeze(0).unsqueeze(0)
  return wav

with torch.no_grad():
  wav_a = load_audio('Prosody2Vec/Emotion Speech Dataset/0011/Happy/0011_000990.wav')
  wav_b = load_audio('Prosody2Vec/Emotion Speech Dataset/0017/Happy/0017_000989.wav')
  enc_out_a = fa_encoder_v2(wav_a)
  prosody_a = fa_encoder_v2.get_prosody_feature(wav_a)
  enc_out_b = fa_encoder_v2(wav_b)
  prosody_b = fa_encoder_v2.get_prosody_feature(wav_b)

  vq_post_emb_a, vq_id_a, _, quantized, spk_embs_a = fa_decoder_v2(
      enc_out_a, prosody_a, eval_vq=False, vq=True
  )
  vq_post_emb_b, vq_id_b, _, quantized, spk_embs_b = fa_decoder_v2(
      enc_out_b, prosody_b, eval_vq=False, vq=True
  )

  vq_post_emb_a_to_b = fa_decoder_v2.vq2emb(vq_id_a, use_residual=False)
  recon_wav_a_to_b = fa_decoder_v2.inference(vq_post_emb_a_to_b, spk_embs_b)

In [99]:
recon_wav_a_to_b = fa_decoder_v2.inference(vq_post_emb_a, spk_embs_b)

In [94]:
ls 'Prosody2Vec/Emotion Speech Dataset'

0001/  0003/  0005/  0007/  0009/  0011/  0013/  0015/  0017/  0019/
0002/  0004/  0006/  0008/  0010/  0012/  0014/  0016/  0018/  0020/


In [100]:
import IPython.display as ipd

ipd.display(ipd.Audio(wav_a.squeeze(0).numpy(), rate=16000))
ipd.display(ipd.Audio(wav_b.squeeze(0).numpy(), rate=16000))
ipd.display(ipd.Audio(recon_wav_a_to_b.detach().squeeze(0).numpy(), rate=16000))
import os